In [85]:
import numpy as np

class RL_system(object):
    
    def __init__(self, _environment):
        self.environment = _environment
        self.learning_vector = initial_learning_vector() # the list of parameters to learn
        self.eps_greedy = 0 # probability to play a random action
        self.discount_factor = 0.1 # specifies how much long term reward is kept
        self.learning_rate = 1 
        
    def learn(self, iterations):
        """ Performs the learning steps a specified number of times, 
            and finally returns the resulting learning_vector. """
        
        for i in iterations:
            learning_step()
            
        return learning_vector
    
    def learning_step(self):
        """ This is the Q-learning routine. """
        
        s = current_state_vector() # current state
        a = policy(s) # action to perform according to the policy
        environment.perform_action(a)
        r = environment.getReward() # gained reward
        update_learning_vector(s,a,r)
    
    def policy(self, s):
        """ Returns the action to perform in the state s according to a policy. """
        
        # two alternatives:
        #  1- Contextual Bandits
        #  2- argmax Q(a) with eps_greedy for exploration
        
        pass
    
    def Q(self, s,a):
        """ This is the Q function, it returns the expected future discounted reward
            for taking action a ∈ A in state s ∈ S. """
        return learning_vector.dot(psi(s,a))
    
    def psi(self, s,a):
        """ Returns a joint representation of state and action. """
        # return np.concatenate(s,a)
        return s # just the state ! 
    
    def update_learning_vector(self, s,a,r):
        """ this function updates the learning vector.
            a is the last performed action,
            s is the previous state,
            r is the reward that has been generated by performing a in state s """
        
        max_Q = max([Q(current_state_vector(),action) for action in environment.getActions()])
        difference = r + discount_factor*max_Q - Q(s,a)
        learning_vector += learning_rate*difference*psi(s,a)
         
            
    def best_Q_policy(self, s):
        """ For a given state It returns the action that maximize the Q_function, but it
            can also return a random action with probability = eps_greedy. """
        actions = environment.getActions()
        
        if random_boolean(eps_greedy):
            return np.random.choice(actions)
        
        i = np.argmax([Q(s,a) for a in actions])
        return actions[i]
    
    
    def random_boolean(self, probability_of_true):
        """ It returns true with the specified probability, false otherwise. """
        return np.random.random_sample()<probability_of_true
    
    def initial_learning_vector(self):
        """ It returns the initial configuration of the learning vector. """
        return np.zeros(len(psi(current_state_vector(),environment.getActions()[0])))
    
    def current_state_vector(self):
        """ It returns the vector of numerical values representing the current state.
            It basically extract the values from the state dictionary of the environment. """
        return np.array(list(environment.getState().values()))
    
    def reset_environment(self):
        environment.restart()
    
    def reset_learning_vector(self):
        learning_vector = initial_learning_vector()

In [84]:
from abc import ABC, abstractmethod

class RL_Environment(ABC):
    
    @abstractmethod
    def getState():
        """ Should return the environment state as a dictionary of (feature name - feature value ). """
        pass
    
    @abstractmethod
    def getActions():
        """ Should return the list of all possible actions. """
        pass
    
    @abstractmethod
    def getReward():
        """ Should return the last reward gained. """
        pass
    
    @abstractmethod
    def perform_action(a):
        """ The environment perform the action a and it's state changes. """
        pass
    
    @abstractmethod
    def restart():
        """ Set the environment to the initial configuration. """
        pass
    
    # add considerations about game end ? 